In [ ]:
# Module Setup and Imports

print("=== Grazioso Rescue Finder CRUD Setup ===")

# Test 1: Import the CRUD module
print("\n1. Testing module import...")
try:
    from CRUD_Python_Module import AnimalShelter
    print("   ✓ CRUD_Python_Module imported successfully")
    print("   ✓ AnimalShelter class available")
except ImportError as e:
    print(f"   ✗ Import failed: {e}")
    print("   ✗ Check that CRUD_Python_Module.py exists and is in the correct directory")
    raise  # Stop execution if import fails
except SyntaxError as e:
    print(f"   ✗ Syntax error in CRUD_Python_Module.py: {e}")
    raise  # Stop execution if there are syntax errors
except Exception as e:
    print(f"   ✗ Unexpected error during import: {e}")
    raise

# Test 2: Instantiate AnimalShelter class
print("\n2. Testing AnimalShelter instantiation...")
try:
    shelter = AnimalShelter()
    print("   ✓ AnimalShelter instance created successfully")
    print("   ✓ MongoDB connection established")
except Exception as e:
    print(f"   ✗ Failed to create AnimalShelter instance: {e}")
    print("   ✗ Check MongoDB server is running and credentials are correct")
    raise  # Stop execution if instantiation fails

print("\n✓ Setup completed successfully - Ready for testing")
print("=" * 60)

In [ ]:
# Test Data Definition - Using actual aac_shelter_outcomes.csv structure

test_animal = {
    "rec_num": "99999",
    "age_upon_outcome": "2 years",
    "animal_id": "TestID001",
    "animal_type": "Dog",
    "breed": "Labrador Retriever Mix",
    "color": "Golden/White",
    "date_of_birth": "2021-06-15",
    "datetime": "2023-06-20 14:30:00",
    "monthyear": "2023-06-20T14:30:00",
    "name": "TestDog",
    "outcome_subtype": "",
    "outcome_type": "Adoption",
    "sex_upon_outcome": "Neutered Male",
    "location_lat": 30.2672,
    "location_long": -97.7431,
    "age_upon_outcome_in_weeks": 104.0
}

# Additional test data for edge cases
empty_id_data = {"animal_id": "", "name": "TestDog", "animal_type": "Dog"}
empty_object = {}

print("✓ Test data defined")
print(f"Valid animal: {test_animal['name']} ({test_animal['animal_id']})")
print(f"Edge case data ready: empty_id_data, empty_object")

In [ ]:
# Test 0: Connection and Authentication Testing

print("=== MongoDB Connection & Authentication Tests ===")

# Test 1: Valid authentication (current connection)
print("\n1. Testing valid authentication...")
try:
    print("   ✓ MongoDB connection successful")
    print(f"   ✓ Connected to database: {shelter._DB}")
    print(f"   ✓ Using collection: {shelter._COL}")
    print(f"   ✓ Authentication successful for user: {shelter._USER}")

    # Test basic operations to verify full functionality
    test_connection = shelter.client.admin.command('ping')
    print(f"   ✓ Server ping successful: {test_connection}")

    # Test connection details
    server_info = shelter.client.server_info()
    print(f"   ✓ MongoDB version: {server_info.get('version', 'Unknown')}")
    print(f"   ✓ Server host: {shelter._HOST}:{shelter._PORT}")

    # Test database access
    db_stats = shelter.database.command('dbStats')
    print(f"   ✓ Database '{shelter._DB}' accessible")
    print(
        f"   ✓ Collections in database: {shelter.database.list_collection_names()}")

except Exception as e:
    print(f"   ✗ Valid authentication failed: {e}")

# Test 2: Invalid authentication
print("\n2. Testing invalid authentication...")
try:
    from pymongo import MongoClient
    from pymongo.errors import ConnectionFailure, ServerSelectionTimeoutError, OperationFailure

    # Test with wrong credentials
    invalid_connection_string = f'mongodb://wronguser:wrongpass@{shelter._HOST}:{shelter._PORT}'

    print("   Testing with invalid credentials (wronguser:wrongpass)...")
    invalid_client = MongoClient(
        invalid_connection_string,
        serverSelectionTimeoutMS=3000,  # Shorter timeout for test
        connectTimeoutMS=3000
    )

    # This should fail when we try to access the database
    invalid_db = invalid_client[shelter._DB]
    invalid_db.command('ping')  # This should raise an authentication error

    print("   ✗ Unexpected: Invalid credentials were accepted!")

except OperationFailure as e:
    if "Authentication failed" in str(e) or "auth" in str(e).lower():
        print("   ✓ Correctly rejected invalid credentials - authentication failed")
    else:
        print(f"   ✓ Connection properly failed: {e}")
except (ConnectionFailure, ServerSelectionTimeoutError) as e:
    print(f"   ✓ Connection properly failed with invalid credentials: {e}")
except Exception as e:
    print(f"   ✓ Authentication properly rejected: {e}")
finally:
    try:
        if 'invalid_client' in locals():
            invalid_client.close()
    except:
        pass

# Test 3: No authentication (if applicable)
print("\n3. Testing connection without authentication...")
try:
    # Test connection without credentials
    no_auth_connection_string = f'mongodb://{shelter._HOST}:{shelter._PORT}'

    print("   Testing connection without credentials...")
    no_auth_client = MongoClient(
        no_auth_connection_string,
        serverSelectionTimeoutMS=3000,
        connectTimeoutMS=3000
    )

    # Try to access the database
    no_auth_db = no_auth_client[shelter._DB]
    no_auth_db.command('ping')

    print("   ⚠ Warning: Database allows unauthenticated access")

except OperationFailure as e:
    if "auth" in str(e).lower() or "unauthorized" in str(e).lower():
        print("   ✓ Correctly requires authentication")
    else:
        print(f"   ✓ Properly secured: {e}")
except (ConnectionFailure, ServerSelectionTimeoutError) as e:
    print(f"   ✓ Connection security working: {e}")
except Exception as e:
    print(f"   ✓ Database properly secured: {e}")
finally:
    try:
        if 'no_auth_client' in locals():
            no_auth_client.close()
    except:
        pass

print("\n✓ Authentication and connection security tests completed")
print("=" * 60)

In [ ]:
# Test 1: Create Operation with Valid Data

print("Testing create operation with valid data...")
create_result = shelter.create(test_animal)

print(f"Create method returned: {create_result}")
print(f"Return type: {type(create_result).__name__}")
print(f"Expected: True (bool)")

if create_result is True:
    print("✓ SUCCESS: Create method returned True as expected")
    print("✓ Record creation completed successfully")
else:
    print(f"✗ FAILURE: Expected True, but got {create_result}")
    print("✗ Create operation did not work as specified")

In [ ]:
# Test 2: Create Operation with None Data (Error Handling Test)

print("Testing create operation with None data...")
none_result = shelter.create(None)

print(f"Create method returned: {none_result}")
print(f"Return type: {type(none_result).__name__}")
print(f"Expected: False (bool)")

if none_result is False:
    print("✓ SUCCESS: Create method returned False as expected")
    print("✓ Properly handled None data")
else:
    print(f"✗ FAILURE: Expected False, but got {none_result}")
    print("✗ None data handling not working as specified")

In [ ]:
# Test 3: Create Operation with Duplicate animal_id (Duplicate Prevention Test)

print("Testing create operation with duplicate animal_id...")
duplicate_result = shelter.create(test_animal)

print(f"Create method returned: {duplicate_result}")
print(f"Return type: {type(duplicate_result).__name__}")
print(f"Expected: False (bool)")

if duplicate_result is False:
    print("✓ SUCCESS: Create method returned False as expected")
    print("✓ Properly prevented duplicate animal_id")
else:
    print(f"✗ FAILURE: Expected False, but got {duplicate_result}")
    print("✗ Duplicate prevention not working as specified")

In [ ]:
# Test 4: Create Operation with Empty animal_id (Validation Test)

print("Testing create operation with empty animal_id...")
empty_id_result = shelter.create(empty_id_data)

print(f"Create method returned: {empty_id_result}")
print(f"Return type: {type(empty_id_result).__name__}")
print(f"Expected: False (bool)")

if empty_id_result is False:
    print("✓ SUCCESS: Create method returned False as expected")
    print("✓ Properly rejected empty animal_id")
else:
    print(f"✗ FAILURE: Expected False, but got {empty_id_result}")
    print("✗ Empty animal_id validation not working as specified")

In [ ]:
# Test 5: Create Operation with Empty Object (Validation Test)

print("Testing create operation with empty object...")
empty_result = shelter.create(empty_object)

print(f"Create method returned: {empty_result}")
print(f"Return type: {type(empty_result).__name__}")
print(f"Expected: False (bool)")

if empty_result is False:
    print("✓ SUCCESS: Create method returned False as expected")
    print("✓ Properly rejected empty object")
else:
    print(f"✗ FAILURE: Expected False, but got {empty_result}")
    print("✗ Empty object validation not working as specified")

In [ ]:
# Test 6: Read Operation with Valid Query (Find Test Record)

print("Testing read operation with valid query...")
read_result = shelter.read({"animal_id": "TestID001"})

print(f"Read method returned: {read_result}")
print(f"Return type: {type(read_result).__name__}")
print(f"Expected: list with 1 record")

if isinstance(read_result, list):
    print(f"✓ SUCCESS: Read method returned list as expected")
    print(f"✓ List contains {len(read_result)} record(s)")

    if len(read_result) > 0:
        found_animal = read_result[0]
        print(f"✓ Record details from database:")
        print(f"   Name: {found_animal.get('name', 'Not found')}")
        print(f"   Breed: {found_animal.get('breed', 'Not found')}")
        print(f"   Animal ID: {found_animal.get('animal_id', 'Not found')}")
        print(f"   Outcome: {found_animal.get('outcome_type', 'Not found')}")
    else:
        print("✗ FAILURE: List is empty - no records found")
else:
    print(f"✗ FAILURE: Expected list, but got {type(read_result).__name__}")
    print("✗ Read operation not returning correct type")

In [ ]:
# Test 7: Read Operation with Non-Matching Query (Empty Results Test)

print("Testing read operation with non-matching query...")
no_match_result = shelter.read({"animal_id": "NonExistentID"})

print(f"Read method returned: {no_match_result}")
print(f"Return type: {type(no_match_result).__name__}")
print(f"Expected: empty list []")

if isinstance(no_match_result, list):
    print(f"✓ SUCCESS: Read method returned list as expected")
    if len(no_match_result) == 0:
        print("✓ SUCCESS: Empty list returned for non-matching query")
        print("✓ Properly handled non-existent record")
    else:
        print(
            f"✗ FAILURE: Expected empty list, but got {len(no_match_result)} record(s)")
else:
    print(
        f"✗ FAILURE: Expected list, but got {type(no_match_result).__name__}")
    print("✗ Read operation not returning correct type")

In [ ]:
# Test 8: Read Operation with None Query (Error Handling Test)

print("Testing read operation with None query...")
none_read_result = shelter.read(None)

print(f"Read method returned: {none_read_result}")
print(f"Return type: {type(none_read_result).__name__}")
print(f"Expected: empty list []")

if isinstance(none_read_result, list):
    print(f"✓ SUCCESS: Read method returned list as expected")
    if len(none_read_result) == 0:
        print("✓ SUCCESS: Empty list returned for None query")
        print("✓ Properly handled None query data")
    else:
        print(
            f"✗ FAILURE: Expected empty list, but got {len(none_read_result)} record(s)")
else:
    print(
        f"✗ FAILURE: Expected list, but got {type(none_read_result).__name__}")
    print("✗ Read operation not returning correct type")

In [ ]:
# Test 9: Error Handling - Connection and Database Operation Tests

print("=== Error Handling Tests ===")

# Test 1: Invalid data types for create operation
print("\n1. Testing create with invalid data types...")
try:
    invalid_types = [
        "string_instead_of_dict",
        123,
        ["list", "instead", "of", "dict"],
        True
    ]

    for invalid_data in invalid_types:
        result = shelter.create(invalid_data)
        print(f"   Invalid type {type(invalid_data).__name__}: {result} ✓")

except Exception as e:
    print(f"   Caught exception with invalid types: {e}")

# Test 2: Invalid data types for read operation
print("\n2. Testing read with invalid data types...")
try:
    invalid_queries = [
        "string_instead_of_dict",
        123,
        ["list", "instead", "of", "dict"],
        True
    ]

    for invalid_query in invalid_queries:
        result = shelter.read(invalid_query)
        print(
            f"   Invalid type {type(invalid_query).__name__}: {len(result)} results ✓")

except Exception as e:
    print(f"   Caught exception with invalid types: {e}")

# Test 3: Malformed document structure
print("\n3. Testing create with malformed document...")
malformed_docs = [
    {"animal_id": "TEST_MALFORMED", "nested": {"invalid": {"deep": "structure"}}},
    {"animal_id": "TEST_SPECIAL_CHARS", "name": "Special!@#$%^&*()Characters"},
    {"animal_id": "TEST_UNICODE", "name": "Unicode™Ñáméś"},
]

for doc in malformed_docs:
    try:
        result = shelter.create(doc)
        print(f"   Malformed doc {doc['animal_id']}: {result}")
        # Clean up if successful
        if result:
            shelter.collection.delete_one({"animal_id": doc["animal_id"]})
    except Exception as e:
        print(f"   Exception with {doc['animal_id']}: {e}")

print("\n✓ Error handling tests completed - all edge cases handled gracefully")

In [ ]:
# Test 10: Update Operation with Valid Query and Update Data (Explicit $set)

print("Testing update operation with explicit $set operator...")

# Update the existing TestID001 record created earlier
update_result = shelter.update(
    {"animal_id": "TestID001"},
    {"$set": {"outcome_type": "Return to Owner"}}
)

print(f"Update method returned: {update_result}")
print(f"Return type: {type(update_result).__name__}")
print(f"Expected: int >= 1")

if isinstance(update_result, int) and update_result >= 1:
    print(f"✓ SUCCESS: Update method returned {update_result} as expected")
    print("✓ Record update completed successfully")

    # Verify the update
    updated_record = shelter.read({"animal_id": "TestID001"})
    if updated_record and updated_record[0].get("outcome_type") == "Return to Owner":
        print("✓ VERIFIED: outcome_type changed to 'Return to Owner'")
else:
    print(f"✗ FAILURE: Expected int >= 1, but got {update_result}")
    print("✗ Update operation did not work as specified")

In [ ]:
# Test 11: Update Operation without Operator (Auto-wrap in $set)

print("Testing update operation without explicit operator (auto-wrap)...")

# Update without $set - should auto-wrap
update_result = shelter.update(
    {"animal_id": "TestID001"},
    {"name": "UpdatedTestDog"}
)

print(f"Update method returned: {update_result}")
print(f"Return type: {type(update_result).__name__}")
print(f"Expected: int >= 1")

if isinstance(update_result, int) and update_result >= 1:
    print(f"✓ SUCCESS: Update method returned {update_result} as expected")
    print("✓ Auto-wrapped update data in $set operator")

    # Verify the update
    updated_record = shelter.read({"animal_id": "TestID001"})
    if updated_record and updated_record[0].get("name") == "UpdatedTestDog":
        print("✓ VERIFIED: name changed to 'UpdatedTestDog'")
else:
    print(f"✗ FAILURE: Expected int >= 1, but got {update_result}")
    print("✗ Auto-wrap functionality not working as specified")

In [ ]:
# Test 12: Update Operation with None Query (Error Handling Test)

print("Testing update operation with None query...")
none_query_result = shelter.update(None, {"outcome_type": "Adoption"})

print(f"Update method returned: {none_query_result}")
print(f"Return type: {type(none_query_result).__name__}")
print(f"Expected: 0 (int)")

if none_query_result == 0:
    print("✓ SUCCESS: Update method returned 0 as expected")
    print("✓ Properly handled None query")
else:
    print(f"✗ FAILURE: Expected 0, but got {none_query_result}")
    print("✗ None query handling not working as specified")

In [ ]:
# Test 13: Update Operation with None Update Data (Error Handling Test)

print("Testing update operation with None update_data...")
none_data_result = shelter.update({"animal_id": "TestID001"}, None)

print(f"Update method returned: {none_data_result}")
print(f"Return type: {type(none_data_result).__name__}")
print(f"Expected: 0 (int)")

if none_data_result == 0:
    print("✓ SUCCESS: Update method returned 0 as expected")
    print("✓ Properly handled None update_data")
else:
    print(f"✗ FAILURE: Expected 0, but got {none_data_result}")
    print("✗ None update_data handling not working as specified")

In [ ]:
# Test 14: Update Operation with Non-Matching Query (No Documents Modified)

print("Testing update operation with non-matching query...")
no_match_update = shelter.update(
    {"animal_id": "NonExistentID999"},
    {"outcome_type": "Adoption"}
)

print(f"Update method returned: {no_match_update}")
print(f"Return type: {type(no_match_update).__name__}")
print(f"Expected: 0 (int)")

if no_match_update == 0:
    print("✓ SUCCESS: Update method returned 0 as expected")
    print("✓ Properly handled non-matching query (no documents modified)")
else:
    print(f"✗ FAILURE: Expected 0, but got {no_match_update}")
    print("✗ Non-matching query handling not working as specified")

In [ ]:
# Test 15: Delete Operation with Valid Query

print("Testing delete operation with valid query...")

# Delete the TestID001 record (updated in previous tests)
delete_result = shelter.delete({"animal_id": "TestID001"})

print(f"Delete method returned: {delete_result}")
print(f"Return type: {type(delete_result).__name__}")
print(f"Expected: int >= 1")

if isinstance(delete_result, int) and delete_result >= 1:
    print(f"✓ SUCCESS: Delete method returned {delete_result} as expected")
    print("✓ Record deletion completed successfully")

    # Verify the deletion
    deleted_check = shelter.read({"animal_id": "TestID001"})
    if len(deleted_check) == 0:
        print("✓ VERIFIED: Record no longer exists in database")
else:
    print(f"✗ FAILURE: Expected int >= 1, but got {delete_result}")
    print("✗ Delete operation did not work as specified")

In [ ]:
# Test 16: Delete Operation with None Query (Error Handling Test)

print("Testing delete operation with None query...")
none_delete_result = shelter.delete(None)

print(f"Delete method returned: {none_delete_result}")
print(f"Return type: {type(none_delete_result).__name__}")
print(f"Expected: 0 (int)")

if none_delete_result == 0:
    print("✓ SUCCESS: Delete method returned 0 as expected")
    print("✓ Properly handled None query")
else:
    print(f"✗ FAILURE: Expected 0, but got {none_delete_result}")
    print("✗ None query handling not working as specified")

In [ ]:
# Test 17: Delete Operation with Non-Matching Query (No Documents Deleted)

print("Testing delete operation with non-matching query...")
no_match_delete = shelter.delete({"animal_id": "NonExistentID999"})

print(f"Delete method returned: {no_match_delete}")
print(f"Return type: {type(no_match_delete).__name__}")
print(f"Expected: 0 (int)")

if no_match_delete == 0:
    print("✓ SUCCESS: Delete method returned 0 as expected")
    print("✓ Properly handled non-matching query (no documents deleted)")
else:
    print(f"✗ FAILURE: Expected 0, but got {no_match_delete}")
    print("✗ Non-matching query handling not working as specified")

In [ ]:
# Test 18: Delete Multiple Documents

print("Testing delete operation with multiple matching documents...")

# Create test records for multi-delete
for i in range(1, 4):
    shelter.create({
        "animal_id": f"TestDelete00{i}",
        "name": f"DeleteDog{i}",
        "outcome_type": "Test_Delete"
    })

print("Created 3 test records")

# Delete all with outcome_type="Test_Delete"
multi_delete = shelter.delete({"outcome_type": "Test_Delete"})

print(f"Delete method returned: {multi_delete}")
print(f"Return type: {type(multi_delete).__name__}")
print(f"Expected: 3 (int)")

if multi_delete == 3:
    print(f"✓ SUCCESS: Deleted {multi_delete} records as expected")
    print("✓ Multiple delete operation successful")
else:
    print(f"✗ FAILURE: Expected 3, but got {multi_delete}")

In [ ]:
# Test Teardown - Cleanup Test Data

print("=== Test Teardown - Cleaning Up Test Data ===")

# Note: TestID001 was already deleted in Test 15
# Note: TestDelete00X records were already deleted in Test 18
cleanup_ids = ["TEST_MALFORMED", "TEST_SPECIAL_CHARS", "TEST_UNICODE"]

print("\nRemoving remaining test records from database...")
for test_id in cleanup_ids:
    try:
        result = shelter.collection.delete_one({"animal_id": test_id})
        if result.deleted_count > 0:
            print(f"   ✓ Removed test record: {test_id}")
        else:
            print(f"   - No record found for: {test_id} (already cleaned up)")
    except Exception as e:
        print(f"   ✗ Error removing {test_id}: {e}")

# Verify cleanup
print("\nVerifying cleanup...")
try:
    remaining_test_records = list(shelter.collection.find({
        "animal_id": {"$in": cleanup_ids}
    }))

    if len(remaining_test_records) == 0:
        print("   ✓ All test records successfully removed")
    else:
        print(f"   ⚠ {len(remaining_test_records)} test records still remain:")
        for record in remaining_test_records:
            print(f"     - {record.get('animal_id', 'Unknown ID')}")

except Exception as e:
    print(f"   ✗ Error verifying cleanup: {e}")

print("\n✓ Test teardown completed")
print("=" * 60)
print("🎉 All tests completed successfully!")
print("📊 Project One CRUD testing suite finished")